# 🧠 Advanced Practice: Variables, Names & Scope in Python

This notebook goes beyond simple assignment and covers:
- Names vs. objects; identity vs. equality
- Mutability, aliasing, and copy semantics
- Unpacking & starred targets; augmented assignment
- Scopes (LEGB), `global`, `nonlocal`, and closures
- Shadowing builtins safely
- `del`, garbage collection cues, and `weakref`
- Variable annotations & type hints
- The walrus operator (`:=`) and comprehension scope

## 1) Names, objects, identity vs equality

In [13]:
a = 256
b = 256
print("a == b:", a == b)
print("a is b:", a is b)

x = 10_000
y = 10_000
print("x == y:", x == y)
print("x is y:", x is y)

s1 = "hello"
s2 = "he" + "llo"
print("s1 is s2:", s1 is s2)

print("id(x):", id(x), "id(y):", id(y))

a == b: True
a is b: True
x == y: True
x is y: False
s1 is s2: True
id(x): 1878908547632 id(y): 1878908547600


## 2) Mutability & aliasing

In [14]:
nums = [1, 2, 3]
alias = nums
alias.append(4)
print("nums:", nums)

import copy
a = [1, [2, 3]]
b = copy.copy(a)
c = copy.deepcopy(a)

a[1].append(99)
print("a:", a)
print("b (shallow):", b)
print("c (deep):", c)

assert a is not b and a == b
assert a is not c and a != c
print("OK ✅")

nums: [1, 2, 3, 4]
a: [1, [2, 3, 99]]
b (shallow): [1, [2, 3, 99]]
c (deep): [1, [2, 3]]
OK ✅


## 3) Unpacking targets, starred assignment, augmented assignment

In [15]:
head, *middle, tail = [10, 20, 30, 40, 50]
print(head, middle, tail)

x = y = [0]
x += [1]
print("x, y:", x, y)

p, q = [], []
p += [1]
print("p, q:", p, q)

10 [20, 30, 40] 50
x, y: [0, 1] [0, 1]
p, q: [1] []


## 4) Scopes (LEGB), `global`, `nonlocal`, closures

In [16]:
name = "module-level"

def outer():
    name = "enclosed"
    def inner():
        # nonlocal name
        # global name
        print("inner sees:", name)
    inner()
    return inner

fn = outer()
fn()
print("module name:", name)

inner sees: enclosed
inner sees: enclosed
module name: module-level


In [17]:
# Closure capturing and late binding pitfall
funcs = []
for i in range(3):
    funcs.append(lambda: i)
print([f() for f in funcs])

funcs2 = []
for i in range(3):
    funcs2.append(lambda i=i: i)
print([f() for f in funcs2])

[2, 2, 2]
[0, 1, 2]


## 5) Shadowing builtins (and how to avoid breaking your session)

In [18]:
import builtins
print("Original len:", len([1,2,3]))

len_backup = builtins.len
try:
    len = 42
    print("len is now:", len)
    print("builtins.len([1,2,3]) =", builtins.len([1,2,3]))
finally:
    if 'len' in globals():
        del globals()['len']
print("Restored len:", len_backup([1,2,3]))

Original len: 3
len is now: 42
builtins.len([1,2,3]) = 3
Restored len: 3


## 6) `del`, garbage collection cues, and `weakref`

In [19]:
import weakref

class Blob:
    def __init__(self, value):
        self.value = value
    def __repr__(self):
        return f"Blob({self.value})"

obj = Blob(99)
ref = weakref.ref(obj)
print("weakref() returns:", ref, "->", ref())

alias = obj
del obj
print("after del obj, weakref ->", ref())
del alias
print("after del alias, weakref ->", ref())

weakref() returns: <weakref at 0x000001B5772222F0; to 'Blob' at 0x000001B577249A90> -> Blob(99)
after del obj, weakref -> Blob(99)
after del alias, weakref -> None


## 7) Variable annotations & static checking hints

In [20]:
from typing import List, Dict, Optional, Tuple

total: int = 0
cache: Dict[str, List[int]] = {}
point: Tuple[int, int] = (10, 20)
nickname: Optional[str] = None

def add_to_cache(key: str, value: List[int]) -> None:
    cache.setdefault(key, []).extend(value)

add_to_cache("a", [1,2,3])
print(cache)

{'a': [1, 2, 3]}


## 8) Walrus operator (`:=`) and comprehension scope

In [21]:
data = ['alice', 'BOB', 'Carla', 'd']
caps = [s for s in data if (n := len(s)) >= 2]
print("caps:", caps)
print("Is 'n' in globals after comprehension?", 'n' in globals())

caps: ['alice', 'BOB', 'Carla']
Is 'n' in globals after comprehension? True


## 9) Mini-exercises (assertions must pass)

In [22]:
funcs = []
for i in range(4):
    funcs.append(lambda i=i: i)
assert [f() for f in funcs] == [0,1,2,3]

a = [1,2,[3]]
b = a.copy()
b[2] = b[2].copy()
a[2].append(4)
assert a != b and a[2] != b[2]

def make_counter(start=0):
    count = start
    def inc():
        nonlocal count
        count += 1
        return count
    return inc
c = make_counter(10)
assert [c(), c(), c()] == [11,12,13]

print('All mini-exercises passed ✅')

All mini-exercises passed ✅
